In [1]:
from VAE import VAE
import sys

sys.path.append("../../")
from utils import load_csv, dataloader, load_save_models
import torch
from torchvision.transforms import v2
from preprocessing.tiff_handling import handle_tiff
import plotly.express as px
from stats import evaluate_perf_utils

In [2]:
batch_size = 436
input_dim = 4800
hidden_dim = 4000
latent_dim = 320

In [3]:
model = VAE(input_dim,hidden_dim,latent_dim)
VAE.load_state_dict(model,state_dict=torch.load('./vae.pth'))

/tmp/ipykernel_82274/2905130577.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  VAE.load_state_dict(model,state_dict=torch.load('./vae.pth'))


<All keys matched successfully>

# load in data necessary for computing the threshold

In [4]:
set = load_csv.load_pandas()
train, val, test = load_csv.split_data(set)

print(f'outlier test validation: {any(val["classification"] != 1)}')

/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/VAE/../../utils/load_csv.py:20: DtypeWarning: Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(data_path, delimiter=";").iloc[:, -3:]


outlier test validation: False


In [5]:
transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize((60, 80)),
        v2.ToTensor(),
        v2.Lambda(
            lambda x: (x.view(-1) - torch.min(x)) / (torch.max(x) - torch.min(x))
        ),
    ]
)

val_set = dataloader.ImagePathDataset(val, transform)

val_dataloader = torch.utils.data.DataLoader(
    val_set, batch_size=batch_size, shuffle=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/home/bendm/machine_learning/project/majority-vote-outlier-detection/.venv/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


VAE(
  (encoder): Sequential(
    (0): Linear(in_features=4800, out_features=4000, bias=True)
    (1): SiLU()
    (2): Linear(in_features=4000, out_features=2000, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2000, out_features=1000, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1000, out_features=500, bias=True)
    (7): SiLU()
    (8): Linear(in_features=500, out_features=640, bias=True)
  )
  (softplus): Softplus(beta=1.0, threshold=20.0)
  (decoder): Sequential(
    (0): Linear(in_features=320, out_features=500, bias=True)
    (1): SiLU()
    (2): Linear(in_features=500, out_features=1000, bias=True)
    (3): SiLU()
    (4): Linear(in_features=1000, out_features=2000, bias=True)
    (5): SiLU()
    (6): Linear(in_features=2000, out_features=4000, bias=True)
    (7): SiLU()
    (8): Linear(in_features=4000, out_features=4800, bias=True)
    (9): Sigmoid()
  )
)

In [6]:
test["image"] = test["file_names"].apply(lambda x: transform(handle_tiff(x)))
test

,classification,file_names,label,image
51,1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.0816), tensor(0.0795), tensor(0.0970..."
6622,1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.1591), tensor(0.1353), tensor(0.1500..."
9400,1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.1166), tensor(0.1164), tensor(0.1190..."
3022,1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.1287), tensor(0.1243), tensor(0.1208..."
1224,1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.1595), tensor(0.1544), tensor(0.1538..."
...,...,...,...,...
153,-1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.0396), tensor(0.0453), tensor(0.0609..."
9812,2,/home/bendm/machine_learning/project/majority-...,Doublet,"[tensor(0.1147), tensor(0.1244), tensor(0.1273..."
7403,2,/home/bendm/machine_learning/project/majority-...,Doublet,"[tensor(0.1007), tensor(0.1204), tensor(0.1291..."
7724,-1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.1171), tensor(0.0756), tensor(0.1007..."


In [17]:
# optimize percentile of error threshold
import optuna

truth = [1 if value == 1 else 0 for value in test["classification"]]


def objective(trial):
    perc = trial.suggest_int("percentile", 0, 100)
    model.extract_error_threshold(
        val_dataloader,
        method="percentile",
        value=perc,
    )
    output = model.predict(test)

    perf_metrics = evaluate_perf_utils.evaluate_performance(truth, output["prediction"])
    return perf_metrics["f1"]


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2024-12-03 16:32:00,751] A new study created in memory with name: no-name-9a475cc7-929a-49d3-94fe-6f1614fb4b44
Computing threshold: 100%|██████████| 4/4 [00:03<00:00,  1.09it/s]
[I 2024-12-03 16:32:06,090] Trial 0 finished with value: 0.5317460317460317 and parameters: {'percentile': 38}. Best is trial 0 with value: 0.5317460317460317.
Computing threshold: 100%|██████████| 4/4 [00:02<00:00,  1.51it/s]
[I 2024-12-03 16:32:10,468] Trial 1 finished with value: 0.6270022883295194 and parameters: {'percentile': 47}. Best is trial 1 with value: 0.6270022883295194.
Computing threshold: 100%|██████████| 4/4 [00:02<00:00,  1.52it/s]
[I 2024-12-03 16:32:14,804] Trial 2 finished with value: 0.5452765540871267 and parameters: {'percentile': 40}. Best is trial 1 with value: 0.6270022883295194.
Computing threshold: 100%|██████████| 4/4 [00:02<00:00,  1.50it/s]
[I 2024-12-03 16:32:19,213] Trial 3 finished with value: 0.706627268889827 and parameters: {'percentile': 58}. Best is trial 3 with value:

In [8]:
print(f"best percentile: {study.best_params['percentile']}")
print(f"best f1: {study.best_value}")

best percentile: 85
best f1: 0.8202431810713112


## perform predictions on the test set

In [18]:
model.extract_error_threshold(val_dataloader, method="percentile", value=85)
output = model.predict(test)
model.threshold

Computing threshold: 100%|██████████| 4/4 [00:03<00:00,  1.14it/s]


np.float64(0.31466254007465516)

## plot out the metrics

In [19]:


truth = [ 1 if value ==1 else 0 for value in  output['classification']]

perf_metrics = evaluate_perf_utils.evaluate_performance(truth, output['prediction'])
tn = perf_metrics['tn']
fp = perf_metrics['fp']
fn = perf_metrics['fn']
tp = perf_metrics['tp']
precision = perf_metrics['precision']
recall = perf_metrics['recall']
f1 = perf_metrics['f1']


perf_metrics

{'tn': np.int64(1119),
 'fp': np.int64(338),
 'fn': np.int64(222),
 'tp': np.int64(1235),
 'precision': np.float64(0.7851239669421488),
 'recall': np.float64(0.8476321207961565),
 'f1': np.float64(0.8151815181518152)}

In [16]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
print(f'accuracy: {accuracy}')

accuracy: 0.8088538091969801


In [12]:
import torch.nn.functional as F

model(test["image"][9997])
px.imshow(test["image"][9997].view(60, 80))

reconstructed = (
    (model(test["image"][9997])["x_reconstructed"]).detach().numpy().reshape(60, 80)
)
px.imshow(reconstructed)
model.eval()
inlier = test["image"][9400]
outlier = test["image"][9997]
inlier_forwards = model(inlier)
outlier_forwards = model(outlier)
inlier_reconstructed = inlier_forwards["x_reconstructed"]
outlier_reconstructed = outlier_forwards["x_reconstructed"]

print(
    f"reconstruction error for inlier: {F.binary_cross_entropy(inlier_reconstructed, inlier,reduction='none').sum(dim=(-1))}"
)
print(
    f"reconstruction error for outlier: {F.binary_cross_entropy(outlier_reconstructed, outlier,reduction='none').sum(dim=(-1))}"
)
# show both images
fig = px.imshow(inlier.view(60, 80))
fig.show()
fig = px.imshow(inlier_reconstructed.detach().numpy().reshape(60, 80))
fig.show()
fig = px.imshow(outlier.view(60, 80))
fig.show()
fig = px.imshow(outlier_reconstructed.detach().numpy().reshape(60, 80))
fig.show()

reconstruction error for inlier: 2138.603515625
reconstruction error for outlier: 2163.463134765625
